In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()

In [2]:
#Importamos la base de datos
from google.colab import files
uploaded = files.upload()

Saving Ventas 2021.xlsx to Ventas 2021.xlsx


# Filtración y trabajo de datos

In [3]:
# LLamamos la base de datos como data
data = pd.read_excel("Ventas 2021.xlsx")

In [4]:
data

,Cuenta,Mes,Clase de documento,Demora tras vencimiento neto,Importe en moneda doc.,Fecha compensación
0,1001053,Enero,DZ,0,-1557899.0,2021-01-06
1,1001053,Enero,DR,49,29662.0,2021-01-06
2,1001053,Enero,DR,46,179827.0,2021-01-06
3,1001053,Enero,DR,45,107290.0,2021-01-06
4,1001053,Enero,DR,40,84391.0,2021-01-06
...,...,...,...,...,...,...
577949,7230233,Septiembre,DZ,771,-1.0,2021-09-28
577950,7605495,Septiembre,AB,0,1.0,2021-09-28
577951,7605495,Septiembre,DZ,790,-1.0,2021-09-28
577952,7607954,Septiembre,AB,0,2.0,2021-09-28


In [5]:
# Eliminamos la columna 'Fecha de compesanción' porque no aporta al análisis
data.drop(['Fecha compensación'], axis = 1, inplace = True)

In [6]:
# Cambiamos el nombre de las columnas
data.rename(
    columns = {"Demora tras vencimiento neto":"Dias de atraso","Importe en moneda doc.":"Importe"
    }, inplace= True)

In [7]:
# Creamos la función que cambia el elemento a un string
def cambio_cuenta(x):
  return str(x)

# Aplicamos la función a la columna 'Cuenta' del dataset data
data['Cuenta_2'] = data['Cuenta'].apply(cambio_cuenta) 

In [8]:
# Eliminamos la columna 'Cuenta'
data.drop(['Cuenta'], axis = 1, inplace = True)

In [9]:
# Cambiamos el orden de las columnas
data = data.reindex(columns=['Cuenta_2','Clase de documento','Dias de atraso','Importe'])

In [10]:
# Eliminamos todos los documentos que no son 'DR' o 'YZ'
df = data[~(data['Clase de documento'] == "DR") & ~(data['Clase de documento'] == "YZ")].index
data.drop(df, inplace=True)

In [11]:
# Eliminamos todos los importes que son <= 0
df1 = data[data['Importe'] <= 0].index
data.drop(df1, inplace= True)

In [12]:
# Re ajustamos el nombre de las variables
data.rename(
    columns = {'Cuenta_2':'Cuenta'}, inplace= True)

In [13]:
data = data.set_index("Cuenta")

In [14]:
data.drop(["C3122","C3126","C3125"], axis = 0, inplace=True)

In [15]:
data.reset_index(inplace=True)

In [16]:
data

,Cuenta,Clase de documento,Dias de atraso,Importe
0,1001053,DR,49,29662.0
1,1001053,DR,46,179827.0
2,1001053,DR,45,107290.0
3,1001053,DR,40,84391.0
4,1001053,DR,32,572256.0
...,...,...,...,...
304824,3339572,DR,0,205880.0
304825,3339575,DR,0,56814.0
304826,3339740,DR,0,125189.0
304827,3339795,DR,0,56620.0


In [17]:
# Podemos ver la cantidad de filas y columnas
data.shape

(304829, 4)

In [18]:
# Vemos un análisis superficial de las variables
data.describe()

,Dias de atraso,Importe
count,304829.000000,3.048290e+05
mean,17.362646,2.253261e+05
std,53.121713,1.292034e+06
min,-116.000000,1.000000e+00
25%,0.000000,4.281600e+04
50%,5.000000,9.172000e+04
75%,20.000000,1.934580e+05
max,1562.000000,2.547802e+08


In [19]:
# Vemos información superficial de las variables
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 304829 entries, 0 to 304828
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Cuenta              304829 non-null  object 
 1   Clase de documento  304829 non-null  object 
 2   Dias de atraso      304829 non-null  int64  
 3   Importe             304829 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 9.3+ MB


# Creación del DataFrame

El DataFrame creado será una agrupación por cuenta y mostrará la siguiente:



1.   Las cuentas de todos los clientes
2.   Los días de atraso promedio de cada cliente
3.   El importe promedio de cada clientes
4.   La probabilidad que posee cada cliente en pagar su deuda en el tiempo establecido



In [20]:
# Creamos una dummy la cual tomará el valor de 1 si el cliente paga dentro del plazo establecido
# y tomará el valor de 0 si el cliente paga atrasado
data_dummies = pd.get_dummies(data['Dias de atraso'] <= 0)

In [21]:
# Agregamos la dummy a nuestro DataSet original
data['Dummy'] = data_dummies[1]

In [22]:
# Ahora agrupamos el DataSet por cuenta y calculamos el promedio de cada variable
data_groupby = round(data.groupby('Cuenta').mean(),2)

# Cambiamos los nombres de las columnas
data_groupby.rename(
    columns = {'Dias de atraso':"Atraso promedio","Importe":"Importe promedio",'Dummy':'Probabilidad de pago'
    }, inplace= True)

# Visualizamos el nuevo DataFrame
data_groupby

# Podemos ver que la columna 'Probabilidad de pago' muestra valores entre 0 y 1. 
# Esto se debe a que es el promedio de las dummies creadas anteriormente, lo que 
# nos revela la probabilidad de pagar a tiempo de los distintos clientes.

,Atraso promedio,Importe promedio,Probabilidad de pago
Cuenta,,,
1000988,10.33,209169.33,0.00
1001000,17.00,137064.00,0.00
1001004,240.14,40312.29,0.14
1001007,2.50,113220.58,0.50
1001010,36.00,94693.45,0.00
...,...,...,...
5500415,17.89,34299.40,0.17
5500423,1.00,8362.00,0.00
5500481,33.71,92968.00,0.00


In [23]:
data_groupby.columns

Index(['Atraso promedio', 'Importe promedio', 'Probabilidad de pago'], dtype='object')

## Avance 16/11/21

En este avance hice:


*   Se creó una columna que cuenta las veces que el cliente realizó una compra
*   Se combinó esa columna en el DF original
*   Se descargó como csv para crear las etiquetas de clasificación



In [24]:
data_count = data.groupby('Cuenta').count()

In [25]:
data_count.drop(['Clase de documento','Dias de atraso','Importe'], axis = 1, inplace = True)

In [26]:
data_count.columns

Index(['Dummy'], dtype='object')

In [27]:
data_count

,Dummy
Cuenta,
1000988,3
1001000,1
1001004,7
1001007,12
1001010,11
...,...
5500415,35
5500423,1
5500481,7


In [28]:
# e.g. save pandas output as csv
#data_groupby.to_csv('Data_trabajada.csv')

In [29]:
#data_t = pd.read_csv('Data_trabajada.csv')

In [30]:
#data_t

In [45]:
combinado = pd.merge(data_groupby, data_count, on='Cuenta', how='left')

In [46]:
combinado

,Atraso promedio,Importe promedio,Probabilidad de pago,Dummy
Cuenta,,,,
1000988,10.33,209169.33,0.00,3
1001000,17.00,137064.00,0.00,1
1001004,240.14,40312.29,0.14,7
1001007,2.50,113220.58,0.50,12
1001010,36.00,94693.45,0.00,11
...,...,...,...,...
5500415,17.89,34299.40,0.17,35
5500423,1.00,8362.00,0.00,1
5500481,33.71,92968.00,0.00,7


In [47]:
combinado.rename(
    columns = {'Dummy':'Contador ventas'}, inplace= True)

In [48]:
combinado.describe()

,Atraso promedio,Importe promedio,Probabilidad de pago,Contador ventas
count,22981.000000,2.298100e+04,22981.000000,22981.000000
mean,17.358722,1.954307e+05,0.318394,13.264392
std,54.841754,6.655522e+05,0.373430,48.258758
min,-88.040000,2.000000e+00,0.000000,1.000000
25%,0.500000,6.193133e+04,0.000000,2.000000
50%,3.520000,1.040080e+05,0.140000,5.000000
75%,16.620000,1.806340e+05,0.600000,12.000000
max,999.000000,5.624249e+07,1.000000,3107.000000


In [36]:
combinado

,Atraso promedio,Importe promedio,Probabilidad de pago,Contador ventas
Cuenta,,,,
1000988,10.33,209169.33,0.00,3
1001000,17.00,137064.00,0.00,1
1001004,240.14,40312.29,0.14,7
1001007,2.50,113220.58,0.50,12
1001010,36.00,94693.45,0.00,11
...,...,...,...,...
5500415,17.89,34299.40,0.17,35
5500423,1.00,8362.00,0.00,1
5500481,33.71,92968.00,0.00,7


In [49]:
combinado = combinado[combinado['Contador ventas'] >= 5]

In [50]:
combinado

,Atraso promedio,Importe promedio,Probabilidad de pago,Contador ventas
Cuenta,,,,
1001004,240.14,40312.29,0.14,7
1001007,2.50,113220.58,0.50,12
1001010,36.00,94693.45,0.00,11
1001028,9.92,145462.83,0.00,12
1001030,52.50,933427.50,0.00,10
...,...,...,...,...
3331698,0.00,114003.80,1.00,5
3333476,2.80,192612.80,0.20,5
5500093,7.30,119913.62,0.00,130


In [51]:
combinado.corr()

,Atraso promedio,Importe promedio,Probabilidad de pago,Contador ventas
Atraso promedio,1.000000,0.028237,-0.407823,0.039475
Importe promedio,0.028237,1.000000,0.013561,0.016686
Probabilidad de pago,-0.407823,0.013561,1.000000,-0.055451
Contador ventas,0.039475,0.016686,-0.055451,1.000000


In [ ]:
combinado.to_csv('Data_trabajada.csv')

## Gráficos

In [ ]:
sns.histplot(data=combinado, x='Contador ventas')
plt.show()

In [ ]:
sns.histplot(data=combinado, x='Contador ventas', binwidth= 1, kde = True)
plt.xlim([0,20])
plt.ylim([0,5000])
plt.show()

In [ ]:
sns.boxplot(data=combinado, x='Atraso promedio')